##### Installing Required Libraries : 

The library being installed is derived from the goes_api library with some modifications and improvements, making it more robust across different operating systems.

In [ ]:
! pip install git+https://github.com/Simoniman/goes_api.git
# ! pip install patool
# ! pip install satpy
# ! pip install rasterio
# ! pip install matplotlib
# ! pip install cartopy 
! wget -N -O utils.py 'https://github.com/Simoniman/GeoSatStitch/raw/main/utils.py' 

##### Importing Required Libraries

In [ ]:
import os
import datetime
from glob import glob
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import goes_api
from goes_api import download_latest_files
from satpy import Scene
from pyresample import create_area_def
from utils import find_format, find_reader, geo_area_def

##### Downloading latest files

To download the latest files, we will use the download_latest_files function. First, we will set the input parameters.

In [ ]:
#### Define protocol and local directory
base_dir = "./tmp/"
protocol = "s3"
fs_args = {}

#### Define satellite, sensor, product_level and product
satellite = "18" # 16 (Goes 16) ==> EAST  /// 18 (Goes 18) ==> WEST
sensor = "ABI"
product_level = "L1B"
product = "Rad"

#### Define sector and filtering options
sector = "F"
scene_abbr = None  # DO NOT SPECIFY FOR FULL DISC SECTOR
scan_modes = None  # select all scan modes (M3, M4, M6)
channels = None  # select all channels
channels = ["C06"]  # select channels subset
filter_parameters = {}
filter_parameters["scan_modes"] = scan_modes
filter_parameters["channels"] = channels
filter_parameters["scene_abbr"] = scene_abbr

#### Downloading options
n_threads = 20  # n_parallel downloads
force_download = False  # whether to overwrite existing data on disk
look_ahead_minutes = 60
N = 1
operational_checks = True

base_dir directory have to be created in advance.

In [ ]:
# Create the base_dir if it doesn't already exist
if not os.path.exists(base_dir):
    os.makedirs(base_dir)
    print(f"Directory '{base_dir}' created successfully.")
else:
    print(f"Directory '{base_dir}' already exists.")

Finally we would run the main function to download the most up-to-date imagery.

In [ ]:
if type(satellite) == list:
  for sat in satellite : 
    fpaths = download_latest_files(
        N=N,
        base_dir=base_dir,
        protocol=protocol,
        fs_args=fs_args,
        satellite=sat,
        sensor=sensor,
        product_level=product_level,
        product=product,
        sector=sector,
        filter_parameters=filter_parameters,
        n_threads=n_threads,
        force_download=force_download,
        check_data_integrity=True,
        progress_bar=True,
        verbose=True,
        look_ahead_minutes=look_ahead_minutes,
    )

else:
    fpaths = download_latest_files(
        N=N,
        base_dir=base_dir,
        protocol=protocol,
        fs_args=fs_args,
        satellite=sat,
        sensor=sensor,
        product_level=product_level,
        product=product,
        sector=sector,
        filter_parameters=filter_parameters,
        n_threads=n_threads,
        force_download=force_download,
        check_data_integrity=True,
        progress_bar=True,
        verbose=True,
        look_ahead_minutes=look_ahead_minutes,
    )

In [ ]:
path = list(fpaths.values())[0][0]
dir = os.path.dirname(path)

##### Reading Data

Satpy's main interface for working with data is the `Scene` class. We can provide the `Scene` with data files and load them with a "reader". 

There are many readers available in Satpy, but the three readers related to this project are as below :

| Reader      | Description |
|:-----------:|:-----------:|
| `'abi_l1b'`              |  reads individual GOES ABI L1B NetCDF4 files. The files read by this reader are described in the [official PUG document](https://www.goes-r.gov/users/docs/PUG-L1b-vol3.pdf)|
| `'seviri_l1b_native'`    |  reads and calibrates MSG-SEVIRI L1.5 image data in binary format. The format is explained in the [MSG Level 1.5 Native Format File Definition.](https://www-cdn.eumetsat.int/files/2020-04/pdf_fg15_msg-native-format-15.pdf)|
|   `'ahi_hsd'`            |  reads Advanced Himawari Imager (AHI) Binary data (standard format data)       |

In [ ]:
base_dir = dir

filenames_format = find_format(base_dir)

reader=find_reader(filenames_format)
filenames = glob( base_dir + f'/*.{filenames_format}')

scn = Scene(reader=reader, filenames=filenames)

We've now created a `Scene` object. Under the hood Satpy has determined what we can access. We haven't actually loaded any data so our dict-like `Scene` object is empty. In other words we first load/read metadata by creating `Scene` object. 

In [ ]:
scn.available_dataset_names()

Then we will read/load all or subset of available dataset. 

In [ ]:
channels = ['IR_108'] 
scn.load(channels)

Now we loaded some dataset and consequently are able to access those.

In [ ]:
scn['IR_108']

##### Resampling Data

One of the more complex topics when it comes to working with earth-observing satellite data is geographic projections and resampling data to these different projections.

The `'area'` attribute of our Satpy data is a special `AreaDefinition` and it defines the geographic area that our data covers. Under `Projection` we see a python dictionary of projection parameters to define that flat plane representation of the Earth. Our ABI data is on a `'geos'` or Geostationary Satellite View coordinate system where position is measured in meters on the the X and Y axes. You can learn more about that on the PROJ site [here](https://proj.org/operations/projections/geos.html).

In the current project we need to have the data in `'web mercator'` projection system(EPSG:3857).
So we need to resample the data to this system.  

We can do this using Satpy's `Scene.resample` method which provides multiple algorithms for resampling data.

- `'native'` resampler can be very useful when you need to combine bands from the same instrument that have different resolutions.

- Other resamplers (`nearest` and so on) are used to change the projection of the dataset. for this case, we should determine the geographic area of interest using a `AreaDefenition` object. this object is created more easily using Pyresample's `create_area_def` utility function.

Pysat utilizes Pyresample, a Python package designed for resampling geospatial image data, for the purpose of resampling operations.

Next we will create `AreaDefenition` object for Goes_East and Goes_West satellites using create_area_def utility function. 

In [ ]:
geo_area_defenition = geo_area_def(scene=scn)

Then we will resample the data using created `AreaDefenition` object. 

In [ ]:
scn_resamp = scn.resample(destination=geo_area_defenition)

##### Writing Data

Satpy makes it possible to save datasets in multiple formats, with `writers` designed to save in a given format.The default `writer` is geotiff writer.

In [ ]:
scn_resamp.save_datasets(
    writer="geotiff",
    filename="{platform_name}_{name}_{start_time:%Y%m%d_%H%M%S}.tif",
    base_dir=base_dir)

##### Visualizing Data

Although data can be visualized in specialized GIS software after exporting the data in well-known geospatial raster formats like GeoTIFF, being able to visualize data directly in Python can be really beneficial in some cases.

In [ ]:
channel = 'C06'
crs = scn[channel].attrs['area'].to_cartopy_crs()
plt.figure(figsize=(16,10))
ax = plt.axes(projection=ccrs.PlateCarree())

cmap = 'Greys'  # Example colormap without transparency

dataset = scn[channel]
dataset.plot.imshow(transform=crs, cmap=cmap)
ax.coastlines()
# ax.gridlines()
ax.gridlines(draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')
plt.title(f"platform_name : {scn[channel].attrs['platform_name']}  \n channel : {scn[channel].attrs['name']}  \n start_time : {scn[channel].attrs['start_time']}  \n end_time : {scn[channel].attrs['end_time']} ")